In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder 
from recbole.config import Config
from recbole.data import create_dataset, data_preparation

from logging import getLogger
from recbole.model.general_recommender import BPR
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger

import torch
from recbole.data.interaction import Interaction

import pickle
import os

In [7]:
def get_history(inter):
    visits = []
    set_uid = set(inter['uid:token'])
    for u in set_uid:
        visits.append(inter[inter['uid:token'] == u]['venue_id:token'].values.tolist()[-100:])
    
    return visits

inter = pd.read_csv('foursquare/foursquare.inter', sep=',')

visits = get_history(inter)

In [8]:
len(set(inter['venue_id:token']))

27898

## Train General Recommendation

In [2]:
# configurations initialization
k = 10

config_dict = {
    #environment settings
    'seed': 1234,
    'reproducibility': True,
    'data_path': os.getcwd(), 
    'topk': k
}

config = Config(model='BPR', dataset='foursquare', config_file_list=['foursquare_general.yaml'], config_dict = config_dict)

# init random seed
init_seed(config['seed'], config['reproducibility'])
# logger initialization
init_logger(config)
logger = getLogger()

# write config info into log
logger.info(config)

# dataset creating and filtering
dataset = create_dataset(config)
logger.info(dataset)

# dataset splitting
train_data, valid_data, test_data = data_preparation(config, dataset)

17 May 09:50    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 1234
state = INFO
reproducibility = True
data_path = /Users/andreafrasson/Desktop/tesi/Feedback-Loop-for-POI-RS/foursquare
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 4096
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'LS': 'valid_and_test'}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 4096

In [4]:
# model loading and initialization
model = BPR(config, train_data.dataset).to(config['device'])
logger.info(model)

17 May 09:01    INFO  BPR(
  (user_embedding): Embedding(1084, 64)
  (item_embedding): Embedding(27899, 64)
  (loss): BPRLoss()
)
Trainable parameters: 1854912


In [5]:
# trainer loading and initialization
trainer = Trainer(config, model)

# model training
best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)

# model evaluation
test_result = trainer.evaluate(test_data)
print(test_result)

17 May 09:01    INFO  epoch 0 training [time: 0.21s, train loss: 17.9465]
17 May 09:01    INFO  epoch 0 evaluating [time: 0.73s, valid_score: 0.290200]
17 May 09:01    INFO  valid result: 
recall@10 : 0.3398    mrr@10 : 0.2902    ndcg@10 : 0.3021    hit@10 : 0.3398    precision@10 : 0.034
17 May 09:01    INFO  Saving current: saved/BPR-May-17-2024_09-01-36.pth
17 May 09:01    INFO  epoch 1 training [time: 0.24s, train loss: 17.6531]
17 May 09:01    INFO  epoch 1 evaluating [time: 0.73s, valid_score: 0.308000]
17 May 09:01    INFO  valid result: 
recall@10 : 0.362    mrr@10 : 0.308    ndcg@10 : 0.3208    hit@10 : 0.362    precision@10 : 0.0362
17 May 09:01    INFO  Saving current: saved/BPR-May-17-2024_09-01-36.pth
17 May 09:01    INFO  epoch 2 training [time: 0.35s, train loss: 17.1217]
17 May 09:01    INFO  epoch 2 evaluating [time: 0.90s, valid_score: 0.318400]
17 May 09:01    INFO  valid result: 
recall@10 : 0.3721    mrr@10 : 0.3184    ndcg@10 : 0.331    hit@10 : 0.3721    precisio

OrderedDict([('recall@10', 0.398), ('mrr@10', 0.3578), ('ndcg@10', 0.3673), ('hit@10', 0.398), ('precision@10', 0.0398)])


## Make prediction

In [29]:
inter = pd.read_csv('foursquare/foursquare.inter', sep='\t')

visits = get_history(inter)

In [30]:
np.array(visits).shape

(1083, 100)

In [66]:
unique_users = list(set(inter['uid:token']))
unique_locations = list(set(inter['venue_id:token']))
print(len(unique_locations), len(unique_users))

28110 1083


In [59]:
current_time = max(inter['timestamp:token'])+1

In [47]:
#make prediction for users
input_inter = Interaction({
    'uid': torch.tensor(unique_users),
    'venue_id': torch.tensor(visits),
    'timestamp': torch.tensor([list(np.arange(current_time))]* len(unique_users))
})

with torch.no_grad():
    scores = model.full_sort_predict(input_inter).reshape((len(unique_users), len(unique_locations)+1))

print(scores.shape)

torch.Size([1083, 27899])


In [48]:
# get the 10 items with highest scores
rec_list = np.argsort(scores, axis = 1)[:, -k:]

In [49]:
# select one item at random for each user
def random_choice(a):
    return int(np.random.choice(a, 1)) + 1

random_item = np.apply_along_axis(random_choice, 1, rec_list)

/var/folders/8_/mkx2sfs13pqgtr19k0_pc89h0000gn/T/ipykernel_1864/3913981734.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return int(np.random.choice(a, 1)) + 1


In [58]:
np.max(random_item)

27886

In [50]:
poi = pd.read_csv('foursquare/foursquare.item', sep = '\t')

with open('saved_dictionary.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

In [51]:
for i in random_item:
    cat = poi[poi['venue_id:token'] == int(i)]['venue_category_name:token'].sample(1).values
    print(cat, loaded_dict[int(cat)])

/var/folders/8_/mkx2sfs13pqgtr19k0_pc89h0000gn/T/ipykernel_1864/3256299709.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(cat, loaded_dict[int(cat)])


[78] Electronics Store
[62] Convenience Store
[141] Medical Center
[53] Clothing Store
[97] French Restaurant
[177] Pizza Place
[22] Bar
[22] Bar
[22] Bar
[86] Fast Food Restaurant
[10] Art Gallery
[94] Food & Drink Shop
[97] French Restaurant
[177] Pizza Place
[3] American Restaurant
[234] Thai Restaurant
[22] Bar
[114] Gym / Fitness Center
[22] Bar
[133] Laundry Service
[161] Neighborhood
[236] Thrift / Vintage Store
[22] Bar
[72] Diner
[170] Park
[194] Restaurant
[184] Post Office
[170] Park
[22] Bar
[94] Food & Drink Shop
[113] Greek Restaurant
[94] Food & Drink Shop
[133] Laundry Service
[165] Office
[65] Cuban Restaurant
[22] Bar
[22] Bar
[35] Building
[3] American Restaurant
[22] Bar
[22] Bar
[123] Hotel
[165] Office
[159] Music Venue
[22] Bar
[39] Caf�
[56] College Academic Building
[78] Electronics Store
[132] Latin American Restaurant
[144] Mexican Restaurant
[199] Sandwich Place
[37] Burrito Place
[199] Sandwich Place
[39] Caf�
[215] Spa / Massage
[10] Art Gallery
[46] Carib

## Add new values in the dataset

In [13]:
current_time = max(inter['timestamp:token'])+1

In [60]:
new_locations = pd.DataFrame({'uid:token': unique_users, 'venue_id:token':random_item.tolist(), 'timestamp:token':[current_time]*len(random_item)}, columns=['uid:token', 'venue_id:token', 'timestamp:token'])

In [62]:
inter = pd.concat([inter, new_locations], axis = 0).reset_index(drop = True)
inter.sort_values(by=['uid:token', 'timestamp:token'], inplace=True)

In [63]:
inter[inter['timestamp:token'] == 100]

,uid:token,venue_id:token,timestamp:token
108300,1,9259,100
108301,2,8466,100
108302,3,22204,100
108303,4,15283,100
108304,5,306,100
...,...,...,...
109378,1079,27836,100
109379,1080,7932,100
109380,1081,27864,100
109381,1082,27886,100


In [64]:
inter.to_csv('foursquare/foursquare.inter', index=False, sep = '\t')